In [ ]:
import os
import sys
import django
from django.conf import settings

# Add the project root to Python path
project_root = '/home/dummyuser/projects/gestral'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Set Django settings module (note: we're in app/ directory, so settings is correct)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'settings')

# Only setup Django if it hasn't been configured yet
if not settings.configured:
    django.setup()

# Import models (we're in app/ directory, so direct import works)
from video_gen.models import VideoProject

# Define async wrapper function
async def test_database_connection():
    """Test database connection in async context"""
    from asgiref.sync import sync_to_async

    try:
        # Wrap QuerySet operations with sync_to_async
        get_count = sync_to_async(lambda: VideoProject.objects.count())
        count = await get_count()
        print(f"Successfully connected! Found {count} video projects.")

        # Get first few projects if any exist
        if count > 0:
            # Get first 3 projects
            get_projects = sync_to_async(lambda: list(VideoProject.objects.all()[:3]))
            first_projects = await get_projects()

            print("\nFirst few projects:")
            for project in first_projects:
                # Check different possible title fields
                title = 'N/A'
                for field_name in ['title', 'name', 'project_name']:
                    if hasattr(project, field_name):
                        title = getattr(project, field_name)
                        break
                print(f"- ID: {project.id}, Title: {title}")
        else:
            print("No projects found in database.")

    except Exception as e:
        print(f"Error connecting to database: {e}")
        print(f"Error type: {type(e).__name__}")
        import traceback
        traceback.print_exc()

# Run the async function
await test_database_connection()

Successfully connected! Found 1 video projects.

First few projects:
- ID: a2c63ce1-041b-45f2-94d5-8dd99dc4aec3, Title: Test


In [3]:
# Helper functions for Django ORM experimentation

async def get_all_models():
    """Get all available models in the project"""
    from django.apps import apps
    from asgiref.sync import sync_to_async

    try:
        get_models = sync_to_async(lambda: apps.get_models())
        all_models = await get_models()
        print("Available models:")
        for model in all_models:
            print(f"- {model._meta.app_label}.{model.__name__}")
    except Exception as e:
        print(f"Error getting models: {e}")

async def inspect_video_project_model():
    """Inspect VideoProject model structure"""
    from video_gen.models import VideoProject
    from asgiref.sync import sync_to_async

    try:
        print("VideoProject model fields:")
        for field in VideoProject._meta.fields:
            print(f"- {field.name}: {field.__class__.__name__}")

        # Also show some sample data structure
        get_first = sync_to_async(lambda: VideoProject.objects.first())
        first_project = await get_first()

        if first_project:
            print(f"\nSample project data:")
            print(f"- ID: {first_project.id}")
            for field in VideoProject._meta.fields:
                if field.name != 'id':
                    value = getattr(first_project, field.name, 'N/A')
                    # Truncate long values
                    if isinstance(value, str) and len(value) > 50:
                        value = value[:50] + "..."
                    print(f"- {field.name}: {value}")
        else:
            print("\nNo projects found to show sample data.")
    except Exception as e:
        print(f"Error inspecting model: {e}")

# Call helper functions
await get_all_models()
print("\n" + "="*50 + "\n")
await inspect_video_project_model()

Available models:
- admin.LogEntry
- auth.Permission
- auth.Group
- auth.User
- contenttypes.ContentType
- sessions.Session
- authtoken.Token
- authtoken.TokenProxy
- account.EmailAddress
- account.EmailConfirmation
- socialaccount.SocialApp
- socialaccount.SocialAccount
- socialaccount.SocialToken
- oauth2_provider.Application
- oauth2_provider.Grant
- oauth2_provider.AccessToken
- oauth2_provider.RefreshToken
- oauth2_provider.IDToken
- user_org.Organization
- user_org.Membership
- user_org.Workspace
- user_org.AnonymousSession
- releases.AppUser
- releases.SourceCodeProvider
- releases.SourceCodeAccount
- releases.Repository
- releases.Product
- releases.ProductRepository
- releases.Release
- releases.SemanticRelease
- video_gen.Media
- video_gen.VideoProjectMedia
- video_gen.BrandAsset
- video_gen.VideoProject
- video_gen.RenderVideo
- video_gen.VideoPipelineRun
- video_gen.Room
- video_gen.Recording
- sound_gen.Genre
- sound_gen.Mood
- sound_gen.License
- sound_gen.Track
- django_